In [1]:
import os
os.listdir(r"C:\Users\arthu\APS360\Project_New\balanced_small_dataset")

['images', 'labels']

In [2]:
# Path to the list_ingredients.txt file
ingredient_list_path = r'C:\Users\arthu\APS360\Project_New\list_ingredients.txt'
# Path to the list_ingredients.txt file
#ingredient_list_path = r'C:\Users\arthu\APS360\Project_New\ML_code\list_ingredients.txt'
# Initialize ingredient lists
ingredients = []
ingredient_frequencies = []

# Read the file and parse ingredients with frequencies
with open(ingredient_list_path, 'r') as f:
    for line in f:
        # Stop reading if we encounter the 'Top 10 Most Common Ingredients' line
        if line.startswith('Top'):
            break

        # Skip lines that are not ingredients or are headers
        if ':' in line or line.strip() == '':
            continue

        # Extract the ingredient name and its frequency
        ingredient = line.split('(')[0].strip()  # Extract ingredient name
        frequency = int(line.split('(')[1].replace(')', '').strip())  # Extract frequency
        ingredients.append(ingredient)
        ingredient_frequencies.append(frequency)

# Check how many ingredients were read and print a few for verification
print(f"Total ingredients: {len(ingredients)}")
print(f"First 10 ingredients: {ingredients[:10]}")  # Display the first 10 ingredients for verification
print(f"First 10 frequencies: {ingredient_frequencies[:10]}")  # Display the first 10 frequencies for verification

# Create a dictionary mapping each ingredient to a unique index (for one-hot encoding)
ingredient_dict = {ingredient: i for i, ingredient in enumerate(ingredients)}

# Create a dictionary mapping each ingredient to its frequency
ingredient_freq_dict = {ingredient: freq for ingredient, freq in zip(ingredients, ingredient_frequencies)}

# Print some examples for verification
print(f"Total unique ingredients mapped: {len(ingredient_dict)}")
print("Example ingredient mapping (index):", list(ingredient_dict.items())[:10])
print("Example ingredient mapping (frequency):", list(ingredient_freq_dict.items())[:10])



Total ingredients: 53
First 10 ingredients: ['agent1', 'agent2', 'allium', 'berries', 'butter1', 'butter2', 'butter3', 'butter4', 'cheese', 'chocolate']
First 10 frequencies: [4337, 4337, 5320, 4501, 3601, 3601, 3601, 3600, 5127, 4310]
Total unique ingredients mapped: 53
Example ingredient mapping (index): [('agent1', 0), ('agent2', 1), ('allium', 2), ('berries', 3), ('butter1', 4), ('butter2', 5), ('butter3', 6), ('butter4', 7), ('cheese', 8), ('chocolate', 9)]
Example ingredient mapping (frequency): [('agent1', 4337), ('agent2', 4337), ('allium', 5320), ('berries', 4501), ('butter1', 3601), ('butter2', 3601), ('butter3', 3601), ('butter4', 3600), ('cheese', 5127), ('chocolate', 4310)]


In [10]:
import os
import random
import torch
import glob
from torch.utils.data import Dataset
from PIL import Image

class IngredientDataset(Dataset):
    def __init__(self, image_folder, label_folder, ingredient_dict, transform=None, folder_percentage=1.0, image_percentage=1.0):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.transform = transform
        self.ingredient_dict = {k.strip().lower(): v for k, v in ingredient_dict.items()}  # Normalize keys
        self.data = []

        # Collect all key names (subfolders in image_folder)
        all_folders = [key_name for key_name in os.listdir(label_folder) if key_name.endswith('.txt')]

        # Sample a subset of folders based on the folder_percentage
        folder_count = int(len(all_folders) * folder_percentage)
        random.seed(42)
        selected_folders = random.sample(all_folders, folder_count)

        for key_name in selected_folders:
            key_name_no_ext = key_name.replace('.txt', '')
            image_dir = os.path.join(image_folder, key_name_no_ext)
            label_file = os.path.join(label_folder, key_name)

            # Check if the image folder and label file both exist
            if not os.path.exists(image_dir):
                print(f"Missing image folder for key: {key_name_no_ext}")
                continue

            if not os.path.exists(label_file):
                print(f"Missing label file for key: {key_name_no_ext}")
                continue

            # Collect all images in the folder
            images = glob.glob(os.path.join(image_dir, "*.jpg"))

            # Debug: Check image count in folder
            print(f"Key: {key_name_no_ext}, Total Images: {len(images)}")

            # Sample a subset of images within the folder based on image_percentage
            image_count = int(len(images) * image_percentage)
            selected_images = random.sample(images, image_count) if image_count < len(images) else images

            # Add each image as a separate entry in the dataset
            for img_path in selected_images:
                self.data.append((key_name_no_ext, img_path, label_file))

        # Debug: Final dataset size
        print(f"Total Dataset Size: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        key_name, img_path, label_path = self.data[idx]

        # Load and transform the image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Load and encode the label
        with open(label_path, 'r') as f:
            ingredients = f.read().strip().split('\n')

        # Debug: Check label file contents
        #print(f"Label File ({label_path}) Contents: {ingredients}")

        label_tensor = torch.tensor(self._encode_ingredients(ingredients), dtype=torch.float)

        # Debug: Check label tensor
        #print(f"Encoded Label Tensor for {key_name}: {label_tensor.nonzero(as_tuple=True)}")

        return key_name, image, label_tensor

    def _encode_ingredients(self, ingredients):
        """Convert list of ingredients to a one-hot vector."""
        ingredient_vector = [0] * len(self.ingredient_dict)
        for ingredient in ingredients:
            normalized_ingredient = ingredient.strip().lower()
            if normalized_ingredient in self.ingredient_dict:
                ingredient_vector[self.ingredient_dict[normalized_ingredient]] = 1
            else:
                # Debug: Log unmapped ingredient
                print(f"Unmapped Ingredient: {ingredient}")
        return ingredient_vector


In [20]:
from torch.utils.data import DataLoader
from torchvision import transforms

# Define image transformations (resize, convert to tensor, normalize)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset paths "C:\Users\arthu\APS360\Project_New\balanced_small_dataset\images\train\05123e5f1b"
#"F:\full_dataset\images"
train_image_folder = r'F:\full_dataset\images\train'
train_label_folder = r'F:\full_dataset\labels\train'
val_image_folder = r'F:\full_dataset\images\val'
val_label_folder = r'F:\full_dataset\labels\val'
test_image_folder = r'F:\full_dataset\images\test'
test_label_folder = r'F:\full_dataset\labels\test'



# Initialize datasets and DataLoaders
train_dataset = IngredientDataset(
    train_image_folder, train_label_folder, ingredient_dict, transform=transform, folder_percentage=1.0, image_percentage=1.0
)
val_dataset = IngredientDataset(
    val_image_folder, val_label_folder, ingredient_dict, transform=transform, folder_percentage=1.0, image_percentage=1.0
)
test_dataset = IngredientDataset(
    test_image_folder, test_label_folder, ingredient_dict, transform=transform, folder_percentage=1.0, image_percentage=1.0
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verify DataLoader
print("Verifying Train Loader...")
for batch_idx, (key_name, image, label) in enumerate(val_loader):
    print(f"Batch {batch_idx + 1}")
    print(f"Key Name: {key_name}")
    print(f"Image Tensor Shape: {image.shape}")
    print(f"Label Tensor Shape: {label.shape}")
    print(f"Non-zero Labels: {label.nonzero(as_tuple=True)}")  # Check for non-zero labels
    break  # Break after the first batch for verification


Key: 3472dad85a, Total Images: 3
Key: 0ba05d0c30, Total Images: 5
Key: 81ebb544e5, Total Images: 28
Key: 741a7c33d4, Total Images: 16
Key: 69d6ab2dcf, Total Images: 26
Key: 41ae05d61a, Total Images: 5
Key: 304376d284, Total Images: 4
Key: 28a08f0371, Total Images: 11
Key: c78add3d31, Total Images: 14
Key: 0edcde1dbf, Total Images: 10
Key: 0dec99d687, Total Images: 11
Key: 2bfa0ff316, Total Images: 12
Key: 67dc0da467, Total Images: 13
Key: 6e76147f1c, Total Images: 7
Key: eeb01ef33d, Total Images: 7
Key: 0c4b2ee781, Total Images: 18
Key: 5dfaadecd6, Total Images: 9
Key: c66dd404bb, Total Images: 5
Key: 6897fe604e, Total Images: 38
Key: d49ce267c0, Total Images: 19
Key: 835a983edc, Total Images: 24
Key: 02f2cf1394, Total Images: 21
Key: 4b93315549, Total Images: 7
Key: c7cfa6559f, Total Images: 12
Key: a049f5bb72, Total Images: 9
Key: 83430d1971, Total Images: 16
Key: 498d8e8a46, Total Images: 54
Key: 663f46975f, Total Images: 10
Key: 9ea30671bb, Total Images: 14
Key: 30238981e0, Total I

In [21]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [22]:
import torch
import torch.nn as nn
from torchvision.models import vit_b_16

class VITIngredientDetector(nn.Module):
   def __init__(self, output_size=100):
       super().__init__()
       # Load pretrained ViT
       self.vit = vit_b_16(pretrained=True)
       
       # Replace classifier head with more sophisticated head
       self.head = nn.Sequential(
           nn.Linear(768, 1024),
           nn.LayerNorm(1024),
           nn.ReLU(),
           nn.Dropout(0.3),
           nn.Linear(1024, 512),
           nn.LayerNorm(512),
           nn.ReLU(),
           nn.Dropout(0.2),
           nn.Linear(512, output_size)
       )
       self.vit.heads = self.head
       
       # Initially freeze all parameters
       for param in self.vit.parameters():
           param.requires_grad = False
           
       # Unfreeze head parameters
       for param in self.head.parameters():
           param.requires_grad = True
           
       # Unfreeze last two transformer layers for fine-tuning
       for param in self.vit.encoder.layers[-2:].parameters():
           param.requires_grad = True
           
   def unfreeze_last_n_layers(self, n):
       """Unfreeze the last n transformer layers"""
       for param in self.vit.encoder.layers[-n:].parameters():
           param.requires_grad = True
           
   def forward(self, x):
       return self.vit(x)


'\nimport torch\nimport torch.nn as nn\nfrom torchvision.models import vit_l_16\n\nclass VITIngredientDetector(nn.Module):\n    def __init__(self, output_size=100):\n        super().__init__()\n        self.vit = vit_l_16(pretrained=True)\n        \n        # Simpler head with stronger regularization\n        self.head = nn.Sequential(\n            nn.Linear(1024, 512),\n            nn.LayerNorm(512),\n            nn.ReLU(),\n            nn.Dropout(0.6),  # Increased dropout\n            nn.Linear(512, output_size)\n        )\n        self.vit.heads = self.head\n        \n        # Freeze more layers\n        for param in self.vit.parameters():\n            param.requires_grad = False\n            \n        # Only unfreeze head\n        for param in self.head.parameters():\n            param.requires_grad = True\n\n    def forward(self, x):\n        return self.vit(x)\n'

In [23]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

def compute_accuracy(y_true, y_pred):
    """
    Compute accuracy as ratio of correctly predicted positive labels to total positive labels
    y_true: ground truth labels
    y_pred: predicted labels
    """
    total_samples = y_true.shape[0]
    accuracies = []
        
    for i in range(total_samples):
        true_positives = np.sum(np.logical_and(y_true[i] == 1, y_pred[i] == 1))
        total_positives = np.sum(y_true[i] == 1)
        if total_positives > 0:  # Avoid division by zero
            accuracies.append(true_positives / total_positives)
        
    return np.mean(accuracies) if accuracies else 0.0


def save_predictions(epoch, labels, predictions, key_names, save_dir="validation_results", samples_per_epoch=10):
    """
    Save validation predictions and labels to text files
    """
    # Create epoch directory
    epoch_dir = os.path.join(save_dir, f"epoch_{epoch+1}")
    os.makedirs(epoch_dir, exist_ok=True)
    
    # Save specified number of samples
    for i in range(min(samples_per_epoch, len(labels))):
        # Get current key and remove any invalid characters
        current_key = str(key_names[i]).replace('/', '_').replace('\\', '_')
        print(f"Sample {i+1} key: {current_key}")  # Debug print
        
        filename = os.path.join(epoch_dir, f"sample_{i+1}_{current_key}.txt")
        
        with open(filename, 'w') as f:
            f.write(f"Sample Key: {current_key}\n\n")  # Add key to file content
            f.write("Ground Truth Labels:\n")
            label_indices = np.where(labels[i] == 1)[0]
            f.write(", ".join([list(ingredient_dict.keys())[idx] for idx in label_indices]))
            f.write("\n\nPredicted Labels:\n")
            
            pred_indices = np.where(predictions[i] == 1)[0]
            f.write(", ".join([list(ingredient_dict.keys())[idx] for idx in pred_indices]))
            
            true_positives = np.sum(np.logical_and(labels[i] == 1, predictions[i] == 1))
            total_positives = np.sum(labels[i] == 1)
            accuracy = true_positives / total_positives if total_positives > 0 else 0
            f.write(f"\n\nSample Accuracy: {accuracy:.4f}")



def custom_multilabel_metrics(y_true, y_pred, class_names):
    """
    Custom metric calculation that consolidates predictions for all label groups
    (e.g., flour1-4, agent1-2, etc.) based on ground truth
    """
    print("Entering custom_multilabel_metrics")
    # Create copies to avoid modifying originals
    y_true_modified = y_true.copy()
    y_pred_modified = y_pred.copy()
    
    # Create a dictionary to group indices by base label
    base_label_groups = {}
    for i, name in enumerate(class_names):
        # Extract base label by removing trailing numbers
        base_label = ''.join(char for char in name if not char.isdigit())
        if base_label not in base_label_groups:
            base_label_groups[base_label] = []
        base_label_groups[base_label].append(i)
    print(base_label_groups)
    
    # For each sample
    for i in range(len(y_true)):
        # Handle each base label group
        for base_indices in base_label_groups.values():
            # Only process groups with multiple variants (len > 1)
            if len(base_indices) > 1:
                # Check if any variant exists in ground truth
                true_variant_indices = [idx for idx in base_indices if y_true[i, idx] == 1]
                if true_variant_indices:
                    true_variant_idx = true_variant_indices[0]  # Get the ground truth variant index
                    # If any variant was predicted, consolidate to the correct one
                    if any(y_pred[i, idx] == 1 for idx in base_indices):
                        # Zero out all variant predictions
                        for idx in base_indices:
                            y_pred_modified[i, idx] = 0
                        # Set the correct variant to 1
                        y_pred_modified[i, true_variant_idx] = 1
    
    # Calculate metrics using modified predictions
    precision = precision_score(y_true_modified, y_pred_modified, average="samples", zero_division=0)
    recall = recall_score(y_true_modified, y_pred_modified, average="samples", zero_division=0)
    f1 = f1_score(y_true_modified, y_pred_modified, average="samples", zero_division=0)
    #accuracy = compute_accuracy(y_true_modified, y_pred_modified)
    
    # Generate classification report
    '''
    class_report = classification_report(y_true_modified, y_pred_modified, 
                                      target_names=class_names, digits=8, 
                                      zero_division=0)
    '''
    return precision, recall, f1


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, class_names=list(ingredient_dict.keys())):
    # At the start of your training script, calculate thresholds
    #frequencies = torch.tensor(list(ingredient_freq_dict.values()), dtype=torch.float)
    #mean_freq = frequencies.mean()
    
    # Track losses and metrics
    train_losses = []
    val_losses = []
    precision_values = []
    recall_values = []
    f1_values = []
    accuracy_values = []  # Add at start with other metric lists

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

    # Warmup settings
    warmup_epochs = 3
    initial_lr = optimizer.param_groups[0]['lr']

    # Move model to the specified device
    model.to(device)
    best_f1 = 0
    best_epoch = 0

    for epoch in range(num_epochs):
        # Warmup logic for learning rate
        if epoch < warmup_epochs:
            lr_scale = min(1.0, float(epoch + 1) / warmup_epochs)
            for pg in optimizer.param_groups:
                pg['lr'] = lr_scale * initial_lr

        # Training phase
        model.train()
        train_loss = 0.0

        print(f"\nEpoch [{epoch + 1}/{num_epochs}]")
        with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
            for batch_idx, (key_name, images, labels) in enumerate(train_loader):
                # Move data to device
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)

                # Calculate false positive indices
                probabilities = torch.sigmoid(outputs)  # Convert logits to probabilities
                preds = (probabilities > 0.35).float()  # Binary predictions
                false_positive_indices = (preds == 1) & (labels == 0)

                # Adjust weights for loss
                weights = torch.ones_like(labels, device=device)  # Initialize weights
                weights[false_positive_indices] *= 2.0  # Penalize false positives more

                # Compute weighted loss
                loss = criterion(outputs, labels)* weights

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.mean().backward()  # Reduce the loss for gradient calculation
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Regularization: Gradient Clipping
                optimizer.step()

                train_loss += loss.mean().item()

                # Update progress bar
                pbar.set_postfix({"Loss": f"{loss.mean().item():.4f}"})
                pbar.update(1)

        train_losses.append(train_loss / len(train_loader))  # Average loss for the epoch

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_preds = []
        val_key_names = []

        with torch.no_grad():
            with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar:
                for batch_idx, (key_name, images, labels) in enumerate(val_loader):
                    val_key_names.extend(key_name)
                    
                    images, labels = images.to(device), labels.to(device)

                    # Forward pass
                    outputs = model(images)

                    # Compute loss
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    # Collect predictions and labels
                    probabilities = torch.sigmoid(outputs)
                    preds = (probabilities > 0.35).float()  # Binary predictions
                    
                    # In your validation loop, replace the prediction line with:
                    #probabilities = torch.sigmoid(outputs)
                    
                    all_labels.append(labels.cpu())
                    all_preds.append(preds.cpu())

                    # Update progress bar
                    pbar.set_postfix({"Val Loss": f"{loss.item():.4f}"})
                    pbar.update(1)

        val_losses.append(val_loss / len(val_loader))  # Average validation loss

        # Flatten label and prediction lists
        all_labels = torch.cat(all_labels, dim=0).numpy()
        all_preds = torch.cat(all_preds, dim=0).numpy()        

        # Compute metrics
        precision, recall, f1= custom_multilabel_metrics(
        all_labels, all_preds, class_names
        )
        
        #precision = precision_score(all_labels, all_preds, average="samples", zero_division=0)
        #recall = recall_score(all_labels, all_preds, average="samples", zero_division=0)
        #f1 = f1_score(all_labels, all_preds, average="samples", zero_division=0)
        #accuracy = compute_accuracy(all_labels, all_preds)

        precision_values.append(precision)
        recall_values.append(recall)
        f1_values.append(f1)
        #accuracy_values.append(accuracy)
        save_predictions(epoch, all_labels, all_preds, val_key_names, samples_per_epoch=10)
        
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            print(f"\nSaving best model with F1: {f1:.4f}")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'f1_score': f1,
                'precision': precision,
                'recall': recall,
                'train_loss': train_losses[-1],
                'val_loss': val_losses[-1]
            }, 'best_model_checkpoint.pth')

        

        # Log epoch summary
        print(f"Epoch Summary - Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        # Class-wise performance
        if class_names is not None:
            class_report = classification_report(all_labels, all_preds, target_names=class_names, digits=8,zero_division=0)
            print("\nClass-wise Performance:")
            print(class_report)

        # Step the scheduler based on F1 score
        scheduler.step(f1)

    # Plot graphs
    plot_training_metrics(train_losses, val_losses, precision_values, recall_values, f1_values, accuracy_values)




def plot_training_metrics(train_losses, val_losses, precision_values, recall_values, f1_values, accuracy_values):
    """Plots training/validation loss and metrics (precision, recall, F1)."""
    epochs = range(1, len(train_losses) + 1)

    # Plot losses
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Train Loss", marker='o')
    plt.plot(epochs, val_losses, label="Validation Loss", marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    # Plot metrics
    plt.subplot(1, 2, 2)
    plt.plot(epochs, precision_values, label="Precision", marker='o')
    plt.plot(epochs, recall_values, label="Recall", marker='o')
    plt.plot(epochs, f1_values, label="F1 Score", marker='o')
    #plt.plot(epochs, accuracy_values, label="Accuracy", marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.title("Precision, Recall and F1 Score")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:

def calculate_aggressive_weights(ingredient_freq_dict):
    """Calculate aggressive class weights based on frequency"""
    frequencies = torch.tensor(list(ingredient_freq_dict.values()), dtype=torch.float)
    total_samples = frequencies.sum()
    
    # Aggressive inverse frequency weighting
    weights = total_samples / (frequencies + 1)
    
    # Apply log scaling to prevent extreme values while maintaining strong weighting
    weights = torch.log1p(weights) * 2  # Multiply by 2 to make weights more aggressive
    
    # Normalize weights but preserve ratios
    weights = weights * (len(frequencies) / weights.sum()) * 3  # Multiply by 3 for stronger weighting
    
    # Clip extremely high weights but allow for more extreme ratios
    weights = torch.clamp(weights, min=1.0, max=25.0)
    
    #print("Weight Statistics:")
    #print(f"Mean weight: {weights.mean():.2f}")
    #print(f"Max weight: {weights.max():.2f}")
    #print(f"Min weight: {weights.min():.2f}")
    
    return weights

model = VITIngredientDetector(output_size=len(ingredient_dict))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Calculate class weights
class_weights = calculate_aggressive_weights(ingredient_freq_dict)
class_weights = class_weights.to(device)

# Initialize criterion
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)

# Train with the two-phase approach
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device, class_names=list(ingredient_dict.keys()))


Epoch [1/20]


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3357/3357 [13:44<00:00,  4.07batch/s, Val Loss=0.4122]


Entering custom_multilabel_metrics
{'agent': [0, 1], 'allium': [2], 'berries': [3], 'butter': [4, 5, 6, 7], 'cheese': [8], 'chocolate': [9], 'cinnamon': [10], 'condiment': [11], 'cream': [12, 13], 'dried fruit, jams, snacks, treats': [14], 'fats': [15], 'flour': [16, 17, 18, 19], 'fruit': [20], 'juice': [21, 22], 'liquids': [23], 'meat and substitutes': [24], 'milk': [25, 26, 27], 'nightshade': [28], 'nut': [29, 30], 'oil': [31, 32, 33], 'pepper': [34, 35], 'powder': [36, 37], 'root vegetable': [38], 'sauce': [39], 'seasoning': [40, 41], 'seed': [42], 'starches': [43], 'syrup': [44, 45, 46], 'vegetable': [47], 'vinegar': [48], 'water': [49, 50, 51], 'wheat product': [52]}
Sample 1 key: af12e593e3
Sample 2 key: af12e593e3
Sample 3 key: af12e593e3
Sample 4 key: af12e593e3
Sample 5 key: af12e593e3
Sample 6 key: af12e593e3
Sample 7 key: af12e593e3
Sample 8 key: af12e593e3
Sample 9 key: af12e593e3
Sample 10 key: af12e593e3

Saving best model with F1: 0.3140
Epoch Summary - Train Loss: 0.576

Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3357/3357 [04:04<00:00, 13.75batch/s, Val Loss=0.4269]


Entering custom_multilabel_metrics
{'agent': [0, 1], 'allium': [2], 'berries': [3], 'butter': [4, 5, 6, 7], 'cheese': [8], 'chocolate': [9], 'cinnamon': [10], 'condiment': [11], 'cream': [12, 13], 'dried fruit, jams, snacks, treats': [14], 'fats': [15], 'flour': [16, 17, 18, 19], 'fruit': [20], 'juice': [21, 22], 'liquids': [23], 'meat and substitutes': [24], 'milk': [25, 26, 27], 'nightshade': [28], 'nut': [29, 30], 'oil': [31, 32, 33], 'pepper': [34, 35], 'powder': [36, 37], 'root vegetable': [38], 'sauce': [39], 'seasoning': [40, 41], 'seed': [42], 'starches': [43], 'syrup': [44, 45, 46], 'vegetable': [47], 'vinegar': [48], 'water': [49, 50, 51], 'wheat product': [52]}
Sample 1 key: af12e593e3
Sample 2 key: af12e593e3
Sample 3 key: af12e593e3
Sample 4 key: af12e593e3
Sample 5 key: af12e593e3
Sample 6 key: af12e593e3
Sample 7 key: af12e593e3
Sample 8 key: af12e593e3
Sample 9 key: af12e593e3
Sample 10 key: af12e593e3

Saving best model with F1: 0.3145
Epoch Summary - Train Loss: 0.570

...

Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3357/3357 [04:06<00:00, 13.63batch/s, Val Loss=0.4516]


Entering custom_multilabel_metrics
{'agent': [0, 1], 'allium': [2], 'berries': [3], 'butter': [4, 5, 6, 7], 'cheese': [8], 'chocolate': [9], 'cinnamon': [10], 'condiment': [11], 'cream': [12, 13], 'dried fruit, jams, snacks, treats': [14], 'fats': [15], 'flour': [16, 17, 18, 19], 'fruit': [20], 'juice': [21, 22], 'liquids': [23], 'meat and substitutes': [24], 'milk': [25, 26, 27], 'nightshade': [28], 'nut': [29, 30], 'oil': [31, 32, 33], 'pepper': [34, 35], 'powder': [36, 37], 'root vegetable': [38], 'sauce': [39], 'seasoning': [40, 41], 'seed': [42], 'starches': [43], 'syrup': [44, 45, 46], 'vegetable': [47], 'vinegar': [48], 'water': [49, 50, 51], 'wheat product': [52]}
Sample 1 key: af12e593e3
Sample 2 key: af12e593e3
Sample 3 key: af12e593e3
Sample 4 key: af12e593e3
Sample 5 key: af12e593e3
Sample 6 key: af12e593e3
Sample 7 key: af12e593e3
Sample 8 key: af12e593e3
Sample 9 key: af12e593e3
Sample 10 key: af12e593e3
Epoch Summary - Train Loss: 0.5553, Val Loss: 0.5229
Precision: 0.31

Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3357/3357 [04:05<00:00, 13.67batch/s, Val Loss=0.4707]


Entering custom_multilabel_metrics
{'agent': [0, 1], 'allium': [2], 'berries': [3], 'butter': [4, 5, 6, 7], 'cheese': [8], 'chocolate': [9], 'cinnamon': [10], 'condiment': [11], 'cream': [12, 13], 'dried fruit, jams, snacks, treats': [14], 'fats': [15], 'flour': [16, 17, 18, 19], 'fruit': [20], 'juice': [21, 22], 'liquids': [23], 'meat and substitutes': [24], 'milk': [25, 26, 27], 'nightshade': [28], 'nut': [29, 30], 'oil': [31, 32, 33], 'pepper': [34, 35], 'powder': [36, 37], 'root vegetable': [38], 'sauce': [39], 'seasoning': [40, 41], 'seed': [42], 'starches': [43], 'syrup': [44, 45, 46], 'vegetable': [47], 'vinegar': [48], 'water': [49, 50, 51], 'wheat product': [52]}
Sample 1 key: af12e593e3
Sample 2 key: af12e593e3
Sample 3 key: af12e593e3
Sample 4 key: af12e593e3
Sample 5 key: af12e593e3
Sample 6 key: af12e593e3
Sample 7 key: af12e593e3
Sample 8 key: af12e593e3
Sample 9 key: af12e593e3
Sample 10 key: af12e593e3

Saving best model with F1: 0.3334
Epoch Summary - Train Loss: 0.550

Validation:  87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                   | 2908/3357 [03:31<00:32, 13.88batch/s, Val Loss=0.4289]

In [ ]:
# Clear GPU memory before training
import torch
import gc

# First delete model and data if they exist
try:
    del model
except:
    pass
try:
    del optimizer
except:
    pass
try:
    del train_loader
except:
    pass
try:
    del val_loader
except:
    pass

# Force CUDA to clear memory
torch.cuda.empty_cache()
gc.collect()
# To check memory usage
print(torch.cuda.memory_allocated()/1024**3, "GB allocated")
print(torch.cuda.memory_reserved()/1024**3, "GB reserved")